## WangchanBERTa-based model



## Load library

In [ ]:
# !pip -q install torch==1.5.0 torchtext==0.4.0 torchvision==0.6.0
!pip -q install torchtext==0.4.0 torchvision==0.6.0
!pip install thai2transformers==0.1.2
!pip install transformers==3.5.1

     |████████████████████████████████| 53 kB 1.3 MB/s 
     |████████████████████████████████| 6.6 MB 6.4 MB/s 
     |████████████████████████████████| 752.0 MB 6.0 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.5.0 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.5.0 which is incompatible.
fastai 2.7.10 requires torchvision>=0.8.2, but you have torchvision 0.6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.5 MB/s 
     |████████████████████████████████| 1.1 MB 72.3 MB/s 
     |████████████████████████████████| 240 kB 67.6 MB/s 
     |████████████████████████████████| 9.6 MB 66.2 MB/s 
     |████████████████████████████████| 4

In [ ]:
!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html

     |████████████████████████████████| 753.4 MB 19 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.6.0 requires torch==1.5.0, but you have torch 1.4.0 which is incompatible.
torchmetrics 0.11.0 requires torch>=1.8.1, but you have torch 1.4.0 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.4.0 which is incompatible.
thai2transformers 0.1.2 requires torch>=1.5.0, but you have torch 1.4.0 which is incompatible.
thai2transformers 0.1.2 requires transformers==3.5.0, but you have transformers 3.5.1 which is incompatible.
pytorch-lightning 1.8.3.post1 requires torch>=1.9.*, but you have torch 1.4.0 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.4.0 which is incompatible.
fastai 2.7.10 requires torchvision>=0.8.2, but you have torchvision 0.6.0 which is incompatible.


In [ ]:
import torch
torch.__version__

'1.4.0'

In [ ]:
import transformers
transformers.__version__

'3.5.1'

In [ ]:
import random
import numpy as np
import os

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed()

Random seed set as 42


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## Load preprocessed datasets

In [ ]:
# from google.colab import files
# files.upload()

Saving Traffy_test_1.csv to Traffy_test_1.csv
Saving Traffy_val_1.csv to Traffy_val_1.csv
Saving Traffy_train.csv to Traffy_train.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#####

import pandas as pd

df_train = pd.read_csv('/content/drive/MyDrive/Traffy_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Traffy_test_1.csv')
df_val = pd.read_csv('/content/drive/MyDrive/Traffy_val_1.csv')

df_train.head()

,Unnamed: 0,message_id,comment,coords,photo,type
0,3268,86390.0,อยากให้ช่วยดูเรื่องทางเท้าในซอยหน่อยค่ะ เข้าใจ...,"['100.50521', '13.69394']",https://storage.googleapis.com/traffy_public_b...,ทางเท้า
1,3692,148606.0,เส้น สุขุมวิทซอย 36 น่ำท่วมสูงมาก และมีกองขยะก...,"['100.57617', '13.71870']",https://storage.googleapis.com/traffy_public_b...,กีดขวาง
2,578,237557.0,อยากให้ช่วยมาจับหมาจรในซอยครับ มีเป็นแก๊งค์ 5 ...,"['100.56333', '13.80616']",https://storage.googleapis.com/traffy_public_b...,สัตว์จรจัด
3,2947,277961.0,มีถังขยะหลายถังเอามาวางหน้าบ้านส่งกลิ่นเหม็น,"['100.70544', '13.81732']",https://storage.googleapis.com/traffy_public_b...,ความสะอาด
4,387,51555.0,สายไฟระโยงระยาง แถวโรงเรียนอัสสัมชัญชาย,"['100.51582', '13.72335']",https://storage.googleapis.com/traffy_public_b...,สายไฟ


In [ ]:
df_train.shape, df_test.shape, df_val.shape

((32562, 6), (6978, 7), (6978, 7))

In [ ]:
df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()
df_val = df_val.drop_duplicates()

In [ ]:
df_train.shape, df_test.shape, df_val.shape

((32562, 6), (6978, 7), (6978, 7))

In [ ]:
df_test.head()

,Unnamed: 0,Unnamed: 0.1,message_id,comment,coords,photo,type
0,13141,4343,152539.0,เจ้าหน้าที่เข้ามาจัดการยกฝาท่อแล้วตามที่เคยแจ้...,"['100.57329', '13.74295']",https://storage.googleapis.com/traffy_public_b...,ท่อระบายน้ำ
1,36,3357,63346.0,ไฟริมถนนบริเวณตั้งแต่แยกเพชรพระราม(ถนนเพชรบุรี...,"['100.52614', '13.75521']",https://storage.googleapis.com/traffy_public_b...,ถนน
2,5093,1448,228500.0,ขยะจำนวนมาก​ เคยแจ้งตั้งแต่มีฟองดดูว์หลายครั้ง...,"['100.59298', '13.78582']",https://storage.googleapis.com/traffy_public_b...,ความสะอาด
3,1202,1468,249090.0,ฝาท่อชำรุด และ ใบไม้อุดท่อ,"['100.66427', '13.68044']",https://storage.googleapis.com/traffy_public_b...,ท่อระบายน้ำ
4,709,1057,191628.0,ต้นไม้บนทางเท้า,"['100.40129', '13.70732']",https://storage.googleapis.com/traffy_public_b...,ต้นไม้


In [ ]:
pd.isnull(df_train).sum()

Unnamed: 0     0
message_id    11
comment       11
coords         0
photo          0
type           0
dtype: int64

In [ ]:
pd.isnull(df_test).sum()

Unnamed: 0      0
Unnamed: 0.1    0
message_id      5
comment         5
coords          0
photo           0
type            0
dtype: int64

In [ ]:
df_test = df_test[~df_test['comment'].isnull()]
df_train = df_train[~df_train['comment'].isnull()]
df_val = df_val[~df_val['comment'].isnull()]

In [ ]:
pd.isnull(df_test).sum(), pd.isnull(df_train).sum()

(Unnamed: 0      0
 Unnamed: 0.1    0
 message_id      0
 comment         0
 coords          0
 photo           0
 type            0
 dtype: int64, Unnamed: 0    0
 message_id    0
 comment       0
 coords        0
 photo         0
 type          0
 dtype: int64)

In [ ]:
#####

train_text = df_train['comment'].tolist()
test_text = df_test['comment'].tolist()
val_text = df_val['comment'].tolist()

val_labels = df_val['type'].tolist()
train_labels = df_train['type'].tolist()
test_labels = df_test['type'].tolist()

In [ ]:
type(train_text)

list

In [ ]:
for text, label in zip(train_text[:5], train_labels[:5]):
  print("Label:", label, "     \t", "Text:", text)

Label: ทางเท้า      	 Text: อยากให้ช่วยดูเรื่องทางเท้าในซอยหน่อยค่ะ เข้าใจว่าความทางเท้าในซอยอาจจะปรับปรุงทั้งหมดยากเพราะพื้นที่น้อย แต่พอมีวิธีที่ทำให้ทางเท้าในซอยสามารถเดินได้ตลอดทางโดยไม่ต้องลงไปเดินบนถนนเสี่ยงอุบะติเหตุมั้ยคะ
Label: กีดขวาง      	 Text: เส้น สุขุมวิทซอย 36 น่ำท่วมสูงมาก และมีกองขยะกีดขวางเยอะมากค่ะ
Label: สัตว์จรจัด      	 Text: อยากให้ช่วยมาจับหมาจรในซอยครับ มีเป็นแก๊งค์ 5 ตัว เคยมีคนถูกกัดแล้วครับ และเกือบโดนกัดหลายคน ตอนนี้แทบไม่กล้าเดินออกไปซื้อของเลยครับ เพราะเกือบโดนกัดหลายรอบแล้ว
Label: ความสะอาด      	 Text: มีถังขยะหลายถังเอามาวางหน้าบ้านส่งกลิ่นเหม็น
Label: สายไฟ      	 Text: สายไฟระโยงระยาง แถวโรงเรียนอัสสัมชัญชาย


## Import library

In [ ]:
# !pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html

In [ ]:
import torch
torch.__version__

'1.4.0'

In [ ]:
# !pip install transformers 

In [ ]:
from transformers import AutoTokenizer

In [ ]:
import torch
import os, sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

from thai2transformers.utils import get_dict_val
from thai2transformers.metrics import classification_metrics, sk_classification_metrics
# from thai2transformers import preprocess
from thai2transformers.preprocess import process_transformers
from thai2transformers.datasets import SequenceClassificationDataset, Task
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset


In [ ]:
from transformers import (
    AutoConfig,
    RobertaConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    CamembertTokenizer,
    pipeline,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

## Converting the dataset into Dataset class


In [ ]:
def perpare_data(texts, labels):
  template = {'text' : [], 'labels' : []}
  for text, label in zip(texts, labels):
    template['text'].append(text)
    template['labels'].append(label)
  return template

train_dataset = Dataset.from_dict(perpare_data(train_text, train_labels))
test_dataset = Dataset.from_dict(perpare_data(test_text, test_labels))
val_dataset = Dataset.from_dict(perpare_data(val_text, val_labels))

## Module initialization

### Data config


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', revision='main')
print(tokenizer('ให้รหัสไวไฟมา ใส่เท่าไหร่ก็เด้งไม่ถูกต้อง'))

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

{'input_ids': [5, 10, 21, 3104, 18794, 26, 10, 221, 1046, 36, 4899, 5727, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(get_dict_val(train_dataset, keys='labels'))
print("Class name \t:",label_encoder.classes_)
print("Class ID \t:", label_encoder.transform(label_encoder.classes_))

Class name 	: ['กีดขวาง' 'คลอง' 'ความปลอดภัย' 'ความสะอาด' 'จราจร' 'ต้นไม้' 'ถนน'
 'ทางเท้า' 'ท่อระบายน้ำ' 'น้ำท่วม' 'สะพาน' 'สัตว์จรจัด' 'สายไฟ']
Class ID 	: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [ ]:
preprocessed_data = { 'train': SequenceClassificationDataset.from_dataset(
                            Task.MULTICLASS_CLS,
                            tokenizer,
                            train_dataset,
                            'text',
                            'labels',
                            max_length=416,
                            bs=1000,
                            preprocessor=process_transformers,
                            prepare_for_tokenization=True,
                            label_encoder=label_encoder),
                        'val': SequenceClassificationDataset.from_dataset(
                            Task.MULTICLASS_CLS,
                            tokenizer,
                            val_dataset,
                            'text',
                            'labels',
                            max_length=416,
                            bs=1000,
                            preprocessor=process_transformers,
                            prepare_for_tokenization=True,
                            label_encoder=label_encoder),
                        }

[INFO] Apply preprocessor to texts.


  0%|          | 0/33 [00:00<?, ?it/s]

0it [00:00, ?it/s]

[INFO] Apply preprocessor to texts.


  0%|          | 0/7 [00:00<?, ?it/s]

0it [00:00, ?it/s]

##### Example of the converted data

In [ ]:
preprocessed_data['train'][0] 

{'input_ids': tensor([    5,  4422,   189,    99,    79,    75,  1548, 11420,  5952,     8,
            10,  2799,    63,    75,  1548, 11420,   585,  1319,   210,   749,
            69,   214,   424,     8,  9274,  7173,  1615,    75,  1548, 11420,
           151,   445, 10233,    75,  4337,  1180,   445,  4216,  2050,  1775,
          6271,   739,   980,  2612,     6]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'label': tensor(7)}

### Model config

In this section, the transformer model (WangchanBERTa) was initialized using the pretrained weights. The description of the model is shown in https://huggingface.co/airesearch/wangchanberta-base-att-spm-uncased.

In [ ]:
model_config = AutoConfig.from_pretrained(
        'airesearch/wangchanberta-base-att-spm-uncased',
         revision='main',
         num_labels=len(label_encoder.classes_)
    )


model_seq_cls = AutoModelForSequenceClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased',
                                                        revision='main',
                                                        config=model_config)

Downloading:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

### Training config
In this section, the training config for the model training such as a scheduler and optimizer was initialized. 

In [ ]:
training_args = TrainingArguments(
                        num_train_epochs=2,
                        max_steps=1500,
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=16,
                        gradient_accumulation_steps=1,
                        learning_rate=3e-5,
                        warmup_steps=75,
                        weight_decay=0.1,
                        adam_epsilon=1e-08,
                        max_grad_norm=1.0,
                        #checkpoint
                        output_dir='./checkpoints/wangchanberta-base-att-spm-uncased/finetuned/ws/',
                        overwrite_output_dir=True,
                        #logs
                        logging_dir='./logs/wangchanberta-base-att-spm-uncased/finetuned/ws/',
                        logging_first_step=False,
                        logging_steps=5,
                        #eval
                        evaluation_strategy='steps',
                        eval_steps=250,
                        load_best_model_at_end=True,
                        #others
                        seed=42,
                        dataloader_drop_last=False,
                        no_cuda=False,
                        metric_for_best_model='f1_macro',
                        prediction_loss_only=False
                )
data_collator = DataCollatorWithPadding(tokenizer, padding=True)


In [ ]:
from google import colab

## Training

The model, data, and training configs were bundled into a `Trainer` class. Then, the training process started.

In [ ]:
trainer = Trainer(
        model=model_seq_cls,
        args=training_args,
        compute_metrics=classification_metrics,
        train_dataset=preprocessed_data['train'],
        eval_dataset=preprocessed_data['val'],
        data_collator=data_collator
    )

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1 Micro,Precision Micro,Recall Micro,F1 Macro,Precision Macro,Recall Macro,Nb Samples
250,0.904382,0.593900,0.848272,0.848272,0.848272,0.848272,0.832797,0.858614,0.828777,6973
500,0.611975,0.284631,0.923993,0.923993,0.923993,0.923993,0.918931,0.923161,0.918548,6973
750,0.064038,0.191403,0.948803,0.948803,0.948803,0.948803,0.947418,0.946530,0.949221,6973
1000,0.145740,0.170229,0.956547,0.956547,0.956547,0.956547,0.955288,0.957014,0.954265,6973
1250,0.281372,0.153697,0.963430,0.963430,0.963430,0.963430,0.961349,0.960984,0.962041,6973
1500,0.221301,0.148694,0.964291,0.964291,0.964291,0.964291,0.962496,0.961746,0.963547,6973


TrainOutput(global_step=1500, training_loss=0.5305121663411458)

In [ ]:
trainer.save_model('./checkpoints/wangchanberta-base-att-spm-uncased/finetuned/checkpoint-best')

## Model Inference

In [ ]:
model_seq_cls_pipeline = pipeline('sentiment-analysis',
                                    model=model_seq_cls,
                                    tokenizer=tokenizer,
                                    device=0) 

In [ ]:
output_mapper = { 'LABEL_{}'.format(idx) : i for idx, i in enumerate(label_encoder.classes_)}
output_mapper

{'LABEL_0': 'กีดขวาง',
 'LABEL_1': 'คลอง',
 'LABEL_2': 'ความปลอดภัย',
 'LABEL_3': 'ความสะอาด',
 'LABEL_4': 'จราจร',
 'LABEL_5': 'ต้นไม้',
 'LABEL_6': 'ถนน',
 'LABEL_7': 'ทางเท้า',
 'LABEL_8': 'ท่อระบายน้ำ',
 'LABEL_9': 'น้ำท่วม',
 'LABEL_10': 'สะพาน',
 'LABEL_11': 'สัตว์จรจัด',
 'LABEL_12': 'สายไฟ'}

In [ ]:
len(val_text)

6973

In [ ]:
# import numpy as np
# a = 'ฟุตบาทหยุบตัว เดินไม่ได้'
# a = a.replace(' ', '<_>')
# a = tokenizer(a, padding = True)
# a = np.array(a['input_ids'])
# a
# a = a['input_ids']
# a = torch.FloatTensor(a)
# a

In [ ]:
y_preds = []
for i in range (len(test_labels)):
  test_text_i = [test_text[i]]
  y_pred = model_seq_cls_pipeline([sentence.replace(' ', '<_>') for sentence in test_text_i])
  y_pred = np.array([output_mapper[i['label']] for i in y_pred])
  y_preds.append(y_pred[0])

In [ ]:
len(y_preds)

6973

In [ ]:
len(test_labels)

6973

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(label_encoder.transform(test_labels), label_encoder.transform(y_preds), digits=4))

              precision    recall  f1-score   support

           0     0.9275    0.9260    0.9268       649
           1     0.9522    0.9300    0.9409       257
           2     0.9276    0.9423    0.9349       381
           3     0.9720    0.9733    0.9726       749
           4     0.9447    0.9447    0.9447       488
           5     0.9191    0.9861    0.9514       288
           6     0.9602    0.9640    0.9621       750
           7     0.9583    0.9493    0.9538       750
           8     0.9896    0.9751    0.9823       683
           9     0.9893    0.9893    0.9893       750
          10     0.9629    0.9725    0.9677       400
          11     0.9940    0.9880    0.9910       167
          12     0.9800    0.9652    0.9726       661

    accuracy                         0.9620      6973
   macro avg     0.9598    0.9620    0.9608      6973
weighted avg     0.9622    0.9620    0.9620      6973



In [ ]:
# random output
for _ in range(5):
  i = np.random.randint(len(test_text))
  print("Input = {}, \nPredicted class = {}".format(test_text[i].replace(' ', '<_>'), y_preds[i]))

Input = ไฟทางติดนะคะ<_>แต่ต้นไม้บังหมดเลย<_>หน้าร้านเรือนรายอ<_>รถเหินหลายคันแล้วค่ะ, 
Predicted class = ต้นไม้
Input = สะพานคลองหนึ่งหักไม่มีทางเข้าออกรบกวนหน่วยงานที่เกี่ยวข้องเข้ามาดำเนินการบรรเทาให้กับชาวบ้านด้วยค่ะ, 
Predicted class = สะพาน
Input = สายไฟหย่อน, 
Predicted class = สายไฟ
Input = มีเศษวัสดุจากการที่มีกลุ่มคนนำรถไปจอดใต้สะพาน, 
Predicted class = สะพาน
Input = 1.<_>ฝาท่อชำรุด<_>บริเวณหน้าสถานที่ทำกิจกรรมเกี่ยวกับเด็ก, 
Predicted class = ท่อระบายน้ำ


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_seq_cls = model_seq_cls.to(device)

In [ ]:
# from transformers import TextClassificationPipeline

# a = 'ฟุตบาทหยุบตัว เดินไม่ได้'
# a = a.replace(' ', '<_>')
# pipe = TextClassificationPipeline(model=model_seq_cls, tokenizer = tokenizer, return_all_scores=True, device = 0)
# pipe(a)